In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#from google.colab import drive
from dateutil.parser import parse
import sys
import os
import json
import pandas as pd
import numpy as np
import csv

# drive.mount('/content/drive', force_remount=True)
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'botsdata' #change this to whatever folder your code is in
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)

# #print(os.listdir(GOOGLE_DRIVE_PATH))

# sys.path.append(GOOGLE_DRIVE_PATH)

original_datasets_path = "./original_datasets/"
final_datasets_path = "./data_csv/final/"
intermediate_path = "./data_csv/intermediate/"

train_data = ['varol-17','cresci-17','pronbots-2019','celebrity-2019','vendor-purchased-2019','botometer-feedback-2019','political-bots-2019']
part_one_files = ['cresci-17','midterm-2018','varol-17']
part_two_files = ['botometer-feedback-2019', 'botwiki-2019','celebrity-2019', 'cresci-rtbust-2019', 'gilani-2017','political-bots-2019','pronbots-2019','vendor-purchased-2019','verified-2019']


Zeroth, human screen name collection part 1

In [36]:
#initialize screen_names dict
screen_names = {"contains_dataset":[]}

#handle cresci data:
screen_names["contains_dataset"].append("cresci-17")
#cresci's only humans are located in the genuine-accounts.csv directory
cresci_genuine_path = f"{original_datasets_path}cresci-17/genuine_accounts.csv/users.csv"
cresci_df = pd.read_csv(cresci_genuine_path)
for screen_name in cresci_df["screen_name"]:
    if screen_name not in screen_names:
        screen_names[screen_name] = 1
    else:
        screen_names[screen_name] += 1

#handle varol data - from the intermediate csv:
screen_names["contains_dataset"].append("varol-17")
varol_intermediate_path = intermediate_path + "varol-17.csv"
varol_df = pd.read_csv(varol_intermediate_path)
for index, row in varol_df.iterrows():
    if row["is_bot"] == 0:
        screen_name = row["screen_name"]
        if screen_name not in screen_names:
            screen_names[screen_name] = 1
        else:
            screen_names[screen_name] += 1

First, human screen name collection part 2

In [37]:
for filename in part_two_files:
  tsvname = os.path.join(original_datasets_path, filename, f"{filename}.tsv")
  jsonname = os.path.join(original_datasets_path, filename, f"{filename}_tweets.json")

  bot_or_human_df = pd.read_csv(tsvname, sep='\t',names = ['id','type'])
  f = open(jsonname)
  data = json.load(f)

  collecting = False
  if (filename not in screen_names["contains_dataset"]) and (filename in train_data):
    collecting = True
    screen_names["contains_dataset"].append(filename)

  for line in data:
    screen_name = line['user']['screen_name']
    user = int(line['user']['id_str'])
    temp_df = bot_or_human_df.loc[bot_or_human_df['id'] == user, 'type']
    if temp_df.size >= 1:
      if (temp_df.iloc[0] == 'human') and collecting:
          if screen_name not in screen_names.keys():
            screen_names[screen_name] = 1
          else:
            screen_names[screen_name] += 1

Second, screen name bigram probability calculations

In [45]:
del screen_names["contains_dataset"]

all_names = screen_names.keys()

#credit to ....
def get_char_ngrams(string, n):
    """
    This function takes a string and a positive integer n as input

    It returns all character n-grams from the string of size n
    They are stored in a list
    """
    char_ngrams = []
    for i in range(len(string) - n + 1):
        char_ngrams.append(string[i:i+n])
    return char_ngrams

def _normalize_helper(dictionary):
    """
    Normalize a single dictionary
    """
    new_dict = {}
    total = sum(dictionary.values())
    for k, v in dictionary.items():
        new_dict[k] = v / total
    return new_dict

def normalize(dictionary):
    """
    This function takes dictionaries in two formats as input:
    1. A dictionary where the keys are strings and the values are numbers
                        OR
    2. A dictionary of dictionaries. The keys of the inner dictionaries should be strings, and the values should be numbers.

    It returns a new dictionary that is normalized such that the values add up to 1, representing a probability distribution. If the input is a dictionary of dictionaries, it normalizes all of the sub-dictionaries.
    """
    if type(list(dictionary.values())[0]) == int:
        return _normalize_helper(dictionary)

    new_dict = {}
    for k, v_dict in dictionary.items():
        new_dict[k] = _normalize_helper(v_dict)
    return new_dict

chars = []
for c in range(ord('A'), ord('Z')+1):
  chars.append(chr(c))
for c in range(ord('a'), ord('z')+1):
  chars.append(chr(c))
for c in range(0,10):
  chars.append(str(c))
chars.append('_')

unigram = {}
conditional = {}
for name in all_names:
  bigrams = get_char_ngrams(name, 2)
  for bigram in bigrams:
    if bigram[0] in unigram.keys():
      unigram[bigram[0]] += 1
    else:
      unigram[bigram[0]] = 1

    if bigram[0] in conditional.keys():
      if bigram[1] in conditional[bigram[0]].keys():
        conditional[bigram[0]][bigram[1]] += 1
      else:
        conditional[bigram[0]][bigram[1]] = 1
    else:
      conditional[bigram[0]] = {}
      conditional[bigram[0]][bigram[1]] = 1


# add 1 smoothing

# for conditional
for c1 in chars:
  if c1 in conditional.keys():
    for c2 in chars:
      if c2 in conditional[c1].keys():
        conditional[c1][c2] += 1
      else:
        conditional[c1][c2] = 1
  else:
    conditional[c1] = {}
    for c2 in chars:
      conditional[c1][c2] = 1

# for unigram
for c in chars:
  if c in unigram.keys():
    unigram[c] += 2*len(chars)
  else:
    unigram[c] = 2*len(chars)


conditional = normalize(conditional)
unigram = normalize(unigram)


bigram_probs = {}
for c1 in chars:
  bigram_probs[c1] = {}
  for c2 in chars:
    bigram_probs[c1][c2] = conditional[c1][c2]*unigram[c1]


def likelihood(screen_name):
  likelihood = 1
  bigram_name = get_char_ngrams(screen_name, 2)
  for bigram in bigram_name:
    likelihood *= bigram_probs[bigram[0]][bigram[1]]
  likelihood = pow(likelihood,1/len(bigram_name))
  return likelihood

Third, part_one_files data preprocessing

In [46]:
columns_order = ["statuses_count", "followers_count", "friends_count", "favourites_count", "listed_count", "default_profile", "profile_use_background_image", "verified", "tweet_freq", "followers_growth_rate", "friends_growth_rate", "favourites_growth_rate", "listed_growth_rate", "followers_friends_ratio", "screen_name_length", "num_digits_in_screen_name", "name_length", "num_digits_in_name", "description_length", "screen_name_likelihood", "is_bot"]
def derive_features(df):
    #get derived features from the 
    df['tweet_freq'] = df.apply(lambda row: row['statuses_count'] / row['age_hr'], axis=1)
    df['followers_growth_rate'] = df.apply(lambda row: row['followers_count'] / row['age_hr'], axis=1)
    df['friends_growth_rate'] = df.apply(lambda row: row['friends_count'] / row['age_hr'], axis=1)
    df['favourites_growth_rate'] = df.apply(lambda row: row['favourites_count'] / row['age_hr'], axis=1)
    df['listed_growth_rate'] = df.apply(lambda row: row['listed_count'] / row['age_hr'], axis=1)
    df['followers_friends_ratio'] = df.apply(lambda row: row['followers_count'] / max(1, row['friends_count']), axis=1)
    df['screen_name_length'] = df.apply(lambda row: len(row['screen_name']), axis=1)
    df['num_digits_in_screen_name'] = df.apply(lambda row: sum(c.isdigit() for c in row['screen_name']), axis=1)
    df['name_length'] = df.apply(lambda row: len(str(row['name'])), axis=1)
    df['num_digits_in_name'] = df.apply(lambda row: sum(c.isdigit() for c in row['name']), axis=1)
    df['description_length'] = df.apply(lambda row: len(row['description']), axis=1)
    df['screen_name_likelihood'] = df.apply(lambda row: likelihood(row['screen_name']), axis=1)


In [47]:
#clean cresci
cresci_dir = os.path.join(original_datasets_path, "cresci-17")
output_path = os.path.join(final_datasets_path, "cresci-17.csv")

assert not os.path.exists(output_path), f"{output_path} already exists"

columns = ["id", "timestamp", "crawled_at", "name", "screen_name", "description", "statuses_count", "followers_count", "friends_count", "favourites_count", "listed_count", "default_profile", "profile_use_background_image", "verified"]
def get_age_hr(row):
    age = parse(str(row['crawled_at'])) -  parse(str(row['timestamp']))

    return age.days * 24 + age.seconds / 3600

for dir in  os.listdir(cresci_dir):
    input_csv = os.path.join(cresci_dir, dir, "users.csv")

    df = pd.read_csv(input_csv)
    df = df[columns]


    df['age_hr'] = df.apply(func = get_age_hr, axis=1)
    df['name'] = df['name'].astype(str)
    df['screen_name'] = df['screen_name'].astype(str)
    df['description'] = df['description'].astype(str)
    df['profile_use_background_image'] = df.apply(lambda row: row['profile_use_background_image'] == 1, axis=1)
    df['default_profile'] = df.apply(lambda row: row['default_profile'] == 1, axis=1)
    df['verified'] = df.apply(lambda row: row['verified'] == 1, axis=1)

    derive_features(df)

    #only genuine is not a bot
    if "genuine" in input_csv:
        df["is_bot"] = 0
    else:
        df["is_bot"] = 1

    df = df[columns_order]
    df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

In [48]:
#clean midterm-18
input_json = os.path.join(original_datasets_path, "midterm-18/midterm-2018_processed_user_objects.json")
input_labels = os.path.join(original_datasets_path, "midterm-18/midterm-2018.tsv")
output_path = os.path.join(final_datasets_path, "midterm-2018.csv")

assert not os.path.exists(output_path), f"{output_path} already exists"

df = pd.read_json(input_json)

def get_age_hr(row):
    age = (parse(str(row['probe_timestamp'])) -  parse(str(row['user_created_at'])))

    return age.days * 24 + age.seconds / 3600

df['age_hr'] = df.apply(func = get_age_hr, axis=1)

df['name'] = df['name'].astype(str)
df['screen_name'] = df['screen_name'].astype(str)
df['description'] = df['description'].astype(str)

derive_features(df)

labels = pd.read_csv(input_labels, delim_whitespace=True, names=["user_id", "is_bot"])

labels["is_bot"] = labels.apply(lambda row: 1 if row["is_bot"] == "bot" else 0, axis=1)

df = df.merge(labels, on="user_id")

df = df[columns_order]

df.to_csv(output_path, index=False)

In [49]:
#derive the features of stream_data and varol from their intermediate csvs
intermediate_csvs = [os.path.join(intermediate_path, "varol-17.csv"), os.path.join(intermediate_path, "stream_users.csv")]
output_paths = [os.path.join(final_datasets_path, "varol-17.csv"), os.path.join(final_datasets_path, "stream_users.csv")]

for input_path, output_path in zip(intermediate_csvs, output_paths):
    df = pd.read_csv(input_path)
    
    def get_age_hr(row):
        age = (parse(str(row['sampled_at'])[:-6]) -  parse(str(row['created_at'])[:-6]))

        return age.days * 24 + age.seconds / 3600

    df['age_hr'] = df.apply(func = get_age_hr, axis=1)

    df['name'] = df['name'].astype(str)
    df['screen_name'] = df['screen_name'].astype(str)
    df['description'] = df['description'].astype(str)

    df['profile_use_background_image'] = df.apply(lambda row: row['banner_url'] != 'N/A', axis=1)

    derive_features(df)

    if "is_bot" in list(df.columns):
        df = df[columns_order]
    else:
        df = df[columns_order[:-1]]
        
    df.to_csv(output_path, index=False)


Fourth, part_two_files data preprocessing

In [50]:
for filename in part_two_files:
  tsvname = os.path.join(original_datasets_path, filename, f"{filename}.tsv")
  jsonname = os.path.join(original_datasets_path, filename, f"{filename}_tweets.json")
  outname =  os.path.join(final_datasets_path, filename + '.csv')

  bot_or_human_df = pd.read_csv(tsvname, sep='\t',names = ['id','type'])
  f = open(jsonname)
  data = json.load(f)

  arr = []
  for line in data:
    row = []
    user = int(line['user']['id_str'])
    # row.append(line['user']['id_str'])
    row.append(line['user']['statuses_count'])
    row.append(line['user']['followers_count'])
    row.append(line['user']['friends_count'])
    row.append(line['user']['favourites_count'])
    row.append(line['user']['listed_count'])
    row.append(line['user']['default_profile'])
    row.append(line['user']['profile_use_background_image'])
    row.append(line['user']['verified'])


    dt_born = parse(line['user']['created_at'])
    dt_curr = parse(line['created_at'])
    age = dt_curr - dt_born
    user_age = age.days*24 + age.seconds/3600

    row.append(line['user']['statuses_count'] / user_age)
    row.append(line['user']['followers_count'] / user_age)
    row.append(line['user']['friends_count'] / user_age)
    row.append(line['user']['favourites_count'] / user_age)
    row.append(line['user']['listed_count'] / user_age)
    row.append(line['user']['followers_count'] / max(1,line['user']['friends_count']))

    screen_name = line['user']['screen_name']
    name = line['user']['name']

    row.append(len(screen_name))
    row.append(sum(c.isdigit() for c in screen_name))
    row.append(len(name))
    row.append(sum(c.isdigit() for c in name))
    row.append(len(line['user']['description']))
    row.append(str(likelihood(screen_name)))



    df2 = bot_or_human_df.loc[bot_or_human_df['id'] == user, 'type']
    if df2.size >= 1:
      if (df2.iloc[0] == 'human'):
        row.append('0')
        arr.append(row)
        
      elif (df2.iloc[0] == 'bot'):
        row.append('1')
        arr.append(row)

  f.close()


  with open(outname, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['statuses_count','followers_count','friends_count','favourites_count','listed_count','default_profile','profile_use_background_image','verified','tweet_freq','followers_growth_rate','friends_growth_rate','favourites_growth_rate','listed_growth_rate', 'followers_friends_ratio', 'screen_name_length','num_digits_in_screen_name', 'name_length','num_digits_in_name','description_length','screen_name_likelihood', 'is_bot'])
    for row in arr:
      writer.writerow(row)

In [54]:
#merge botwiki and verified
botwiki_df = pd.read_csv(os.path.join(final_datasets_path, "botwiki-2019.csv"))
verified_df = pd.read_csv(os.path.join(final_datasets_path, "verified-2019.csv"))

pd.concat([botwiki_df, verified_df], ignore_index=True).to_csv(os.path.join(final_datasets_path, "botwiki-verified.csv"), index=False)